# Titulo

## Desenvolupament
### 1. EDA (exploratory data analysis)
Com a primer pas, caldrà que importem totes les llibreries i utilitats que farem servir en el desenvolupament de la pràctica:

In [3]:
# Import libraries 

%pip install pandas seaborn scikit-learn category_encoders scikit-optimize scipy openpyxl

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.cm import gist_heat_r
import pandas as pd
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore')

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\arnau\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


A continuació, farem la importació dels datasets (arxius d'extensió **.xlsx**) a analitzar i els analitzarem breument:

In [4]:
dataframe_bancs_per_provincia = pd.read_excel('data/Bancs per provincia.xlsx', sheet_name=0, header=0, engine='openpyxl')
print("Estructura Interna de dataframe_bancs_per_provincia: ", dataframe_bancs_per_provincia.shape)

# View complete dataframe with print(dataframe_bancs_per_provincia)
print(dataframe_bancs_per_provincia.head(5))

Estructura Interna de dataframe_bancs_per_provincia:  (53, 5)
     Columna1          Columna2            Columna3                Columna4  \
0  Provincia   Banco de España   Oficinas en España  Entidades de depósito    
1       Álava               NaN                 147                     147   
2    Albacete               NaN                 211                     211   
3     Alacant                 1                 603                     601   
4     Almería               NaN                 286                     285   

                            Columna5  
0  Otras entidades de crédito y EFC   
1                                NaN  
2                                NaN  
3                                  2  
4                                  1  


In [5]:
dataframe_empresa_per_mida_and_provincia = pd.read_excel('data/Empresa per mida y provincia 2008-2022.xlsx', sheet_name=0, header=0, engine='openpyxl')
print("Estructura Interna de dataframe_empresa_per_mida_and_provincia: ", dataframe_empresa_per_mida_and_provincia.shape)

print(dataframe_empresa_per_mida_and_provincia.head(5))

Estructura Interna de dataframe_empresa_per_mida_and_provincia:  (72, 31)
   Unnamed: 0       CCAA Provincias  2009 Total      2009  2010 Total  \
0         NaN  Andalucía    Almería       42535  0.085366       41548   
1         NaN  Andalucía      Cádiz       61021  0.122467       60125   
2         NaN  Andalucía    Córdoba       48222  0.096780       47422   
3         NaN  Andalucía    Granada       58317  0.117040       57618   
4         NaN  Andalucía     Huelva       26263  0.052709       25795   

       2010  2011 Total      2011  2012 Total  ...  2018 Total      2018  \
0  0.084446       40995  0.085065       40516  ...       43406  0.083709   
1  0.122204       58960  0.122342       57288  ...       61786  0.119156   
2  0.096385       46553  0.096598       45278  ...       47874  0.092326   
3  0.117109       56209  0.116634       55183  ...       60020  0.115750   
4  0.052428       24981  0.051836       24168  ...       25236  0.048668   

   2019 Total      2019  2020 

In [6]:
dataframe_variacio_PIB_per_capita = pd.read_excel('data/Variació PIB per capita.xlsx', sheet_name=0, header=0, engine='openpyxl')
print("Estructura Interna de dataframe_variacio_PIB_per_capita: ", dataframe_variacio_PIB_per_capita.shape)

print(dataframe_variacio_PIB_per_capita.head(5))


Estructura Interna de dataframe_variacio_PIB_per_capita:  (62, 15)
         AÑO   2009   2010   2011   2012   2013   2014   2015   2016   2017  \
0  ANDALUCÍA  17685  17567  17299  16597  16381  16600  17458  17865  18691   
1    Almería  19151  18616  17294  17031  16759  17515  18724  20196  21467   
2      Cádiz  16899  17054  16881  16196  15843  15703  16255  16784  17584   
3    Córdoba  16816  16641  16403  15479  15710  15846  16862  17197  17967   
4    Granada  16639  16431  16347  15971  15877  16429  17320  17368  17846   

    2018   2019   2020   2021   2022  
0  19291  19744  17772  19591  21532  
1  21567  22268  20335  20292  22078  
2  18079  18767  16418  18313  20520  
3  18275  18225  16851  18710  20022  
4  18353  19091  17310  18776  20623  


In [7]:
dataframe_variacio_PIB_percentatge = pd.read_excel('data/Variació PIB porcentaje.xlsx', sheet_name=0, header=0, engine='openpyxl')
print("Estructura Interna de dataframe_variacio_PIB_percentatge: ", dataframe_variacio_PIB_percentatge.shape)

print(dataframe_variacio_PIB_percentatge.head(5))

Estructura Interna de dataframe_variacio_PIB_percentatge:  (63, 15)
         AÑO      2009      2010      2011      2012      2013      2014  \
0  ANDALUCÍA -0.042334  0.000354 -0.009320 -0.037661 -0.012646  0.013410   
1    Almería -0.073423 -0.017927 -0.066082 -0.011342 -0.014347  0.045880   
2      Cádiz -0.060030  0.016432 -0.003580 -0.038539 -0.021253 -0.009186   
3    Córdoba -0.036448 -0.007537 -0.012704 -0.057629  0.011554  0.004983   
4    Granada -0.042923 -0.006667 -0.001451 -0.022298 -0.007408  0.032176   

       2015      2016      2017      2018      2019      2020      2021  \
0  0.052258  0.023877  0.045947  0.032558  0.028317 -0.096263  0.105305   
1  0.076989  0.087118  0.068638  0.008094  0.047211 -0.077435  0.006557   
2  0.034527  0.032223  0.046976  0.027549  0.039619 -0.123122  0.117175   
3  0.059463  0.015954  0.040222  0.012709 -0.003859 -0.077453  0.104886   
4  0.052094  0.001488  0.026819  0.029684  0.044019 -0.089531  0.087010   

       2022  
0  0.10676

In [8]:
dataframe_variacio_PIB = pd.read_excel('data/Variació PIB.xlsx', sheet_name=0, header=0, engine='openpyxl')
print("Estructura Interna de dataframe_variacio_PIB_percentatge: ", dataframe_variacio_PIB_percentatge.shape)

print(dataframe_variacio_PIB_percentatge.head(5))

Estructura Interna de dataframe_variacio_PIB_percentatge:  (63, 15)
         AÑO      2009      2010      2011      2012      2013      2014  \
0  ANDALUCÍA -0.042334  0.000354 -0.009320 -0.037661 -0.012646  0.013410   
1    Almería -0.073423 -0.017927 -0.066082 -0.011342 -0.014347  0.045880   
2      Cádiz -0.060030  0.016432 -0.003580 -0.038539 -0.021253 -0.009186   
3    Córdoba -0.036448 -0.007537 -0.012704 -0.057629  0.011554  0.004983   
4    Granada -0.042923 -0.006667 -0.001451 -0.022298 -0.007408  0.032176   

       2015      2016      2017      2018      2019      2020      2021  \
0  0.052258  0.023877  0.045947  0.032558  0.028317 -0.096263  0.105305   
1  0.076989  0.087118  0.068638  0.008094  0.047211 -0.077435  0.006557   
2  0.034527  0.032223  0.046976  0.027549  0.039619 -0.123122  0.117175   
3  0.059463  0.015954  0.040222  0.012709 -0.003859 -0.077453  0.104886   
4  0.052094  0.001488  0.026819  0.029684  0.044019 -0.089531  0.087010   

       2022  
0  0.10676